#### order brushing
- 先找出 deemed shop
- 再找出該 shop 下的 deemed buyers


In [1]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
def gen_dayHrMin(user_clicks, time_col):
    # # 將 日-時-分 拆出來
#     user_clicks['year'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[0]))
#     user_clicks['month'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
    user_clicks['day'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[2]))
    user_clicks['hour'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[0]))
    user_clicks['minute'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[1]))
    user_clicks['second'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[2]))
#     user_clicks.drop(time_col, axis=1, inplace=True)
    return user_clicks

def gen_timeblock(df):
    df['timeblock'] = (df['day'] - 27) * 24 + df['hour']
    df['min_block'] = (df['day'] - 27) * 24 * 60 + df['hour'] * 60 + df['minute']
    df['sec_block'] = (df['day'] - 27) * 24 * 3600 + df['hour'] * 3600 + df['minute'] * 60 + df['second']
    df = df.sort_values('sec_block')
    return df


In [3]:
df = pd.read_csv('order_brush_order.csv')
df.isnull().sum()
len(df)
df['orderid'] = df['orderid'].astype(str)
df['shopid'] = df['shopid'].astype(str)
df['userid'] = df['userid'].astype(str)
df = gen_dayHrMin(df, 'event_time')
df = gen_timeblock(df)
df.head()


orderid       0
shopid        0
userid        0
event_time    0
dtype: int64

222750

,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,27,0,0,2,0,0,2
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,27,0,0,5,0,0,5


In [5]:
deemed_records_3 = list()
for shop in tqdm(df.shopid.unique()[:3]):
    tmp = df[df['shopid']==shop]
    tmp
    for sec in tqdm(tmp.sec_block.unique()):
        print(sec, sec+3600)
        tmp = tmp[(tmp['sec_block']>=sec)&(tmp['sec_block']<=sec+3600)].sort_values('sec_block')
        tmp
        if len(tmp) > 0:
            concentrate_rate = len(tmp.orderid.values) / len(tmp.userid.unique())
            if concentrate_rate >= 3:
                print(shop, sec, sec+3600, concentrate_rate)
                protion_df = tmp.groupby(['userid'])['orderid'].count().reset_index()
                protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
                protion_df
                deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
                for deemed_buyer in deemed_buyers:
                    deemed_records_2.append([shop, deemed_buyer])


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
184336,31161864165194,6042309,214499001,2019-12-28 00:04:25,28,0,4,25,24,1444,86665
62383,31200635667837,6042309,104742009,2019-12-28 10:50:35,28,10,50,35,34,2090,125435
216319,31207959482690,6042309,23192275,2019-12-28 12:52:39,28,12,52,39,36,2212,132759
52088,31220731156583,6042309,75901,2019-12-28 16:25:31,28,16,25,31,40,2425,145531
148962,31223079165230,6042309,75606749,2019-12-28 17:04:39,28,17,4,39,41,2464,147879
62380,31223223812899,6042309,75606749,2019-12-28 17:07:04,28,17,7,4,41,2467,148024
120199,31229316246138,6042309,34132265,2019-12-28 18:48:36,28,18,48,36,42,2568,154116
128348,31232611761617,6042309,9599497,2019-12-28 19:43:31,28,19,43,31,43,2623,157411
219656,31301129203990,6042309,99531028,2019-12-29 14:45:30,29,14,45,30,62,3765,225930



  0%|                                                                                           | 0/26 [00:00<?, ?it/s]

0 3600


,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0


...

428523 432123


,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block


431620 435220


,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.82it/s]


In [94]:
# output deemed_records
deemed_records = list()
for shop in tqdm(df.shopid.unique()):
    tmp = df[df['shopid']==shop].sort_values('event_time')
    for block in tmp.timeblock.unique():
        concentrate_rate = len(tmp[tmp['timeblock']==block].orderid.values) / len(tmp[tmp['timeblock']==block].userid.unique())
        if concentrate_rate >= 3:
            print(shop, block, concentrate_rate)
            protion_df = tmp[(tmp['timeblock']==block)|(tmp['timeblock']==block+1)].groupby(['userid'])['orderid'].count().reset_index()
            protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
            deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
            for deemed_buyer in deemed_buyers:
                deemed_records.append([shop, deemed_buyer])
                 
        if concentrate_rate >= 3:
            print(shop, block, block+1, concentrate_rate)
            protion_df = tmp[(tmp['timeblock']==block)|(tmp['timeblock']==block+1)].groupby(['userid'])['orderid'].count().reset_index()
            protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
            deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
            for deemed_buyer in deemed_buyers:
                deemed_records.append([shop, deemed_buyer])




  0%|                                                                                        | 0/18770 [00:00<?, ?it/s]

  0%|                                                                                | 1/18770 [00:00<50:20,  6.21it/s]

  0%|                                                                              | 2/18770 [00:00<1:04:16,  4.87it/s]

  0%|                                                                                | 6/18770 [00:00<47:26,  6.59it/s]

  0%|                                                                                | 8/18770 [00:00<37:59,  8.23it/s]

  0%|                                                                               | 10/18770 [00:00<31:55,  9.79it/s]

  0%|                                                                               | 12/18770 [00:00<27:39, 11.30it/s]

  0%|                                                                               | 14/18770 [00:01<30:30, 10.25it/s]

  0%|                         

18190345 52 3.0
18190345 52 3.0




  0%|                                                                               | 25/18770 [00:02<39:51,  7.84it/s]

  0%|                                                                               | 27/18770 [00:02<42:13,  7.40it/s]

  0%|                                                                               | 29/18770 [00:02<40:02,  7.80it/s]

  0%|▏                                                                              | 30/18770 [00:03<43:54,  7.11it/s]

  0%|▏                                                                              | 32/18770 [00:03<37:47,  8.26it/s]

  0%|▏                                                                              | 33/18770 [00:03<48:51,  6.39it/s]

  0%|▏                                                                              | 35/18770 [00:03<42:53,  7.28it/s]

  0%|▏                                                                              | 36/18770 [00:03<45:08,  6.92it/s]

  0%|▏                        

27476241 9 5.0
27476241 10 4.0




  0%|▏                                                                              | 46/18770 [00:04<32:35,  9.58it/s]

  0%|▏                                                                              | 48/18770 [00:04<31:16,  9.98it/s]

  0%|▏                                                                              | 51/18770 [00:05<28:06, 11.10it/s]

  0%|▏                                                                              | 53/18770 [00:05<31:04, 10.04it/s]

  0%|▏                                                                              | 56/18770 [00:05<28:19, 11.01it/s]

  0%|▏                                                                              | 58/18770 [00:05<27:11, 11.47it/s]

  0%|▎                                                                              | 60/18770 [00:05<25:04, 12.43it/s]

  0%|▎                                                                              | 62/18770 [00:06<24:52, 12.54it/s]

  0%|▎                        

...



 96%|████████████████████████████████████████████████████████████████████████▋   | 17963/18770 [11:36<00:18, 42.54it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17968/18770 [11:36<00:18, 43.51it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17973/18770 [11:36<00:18, 43.56it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17978/18770 [11:36<00:18, 43.85it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17983/18770 [11:36<00:17, 44.28it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17988/18770 [11:36<00:17, 44.77it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17993/18770 [11:36<00:17, 44.81it/s]

 96%|████████████████████████████████████████████████████████████████████████▊   | 17998/18770 [11:37<00:17, 44.54it/s]

 96%|█████████████████████████

123401085 115 4.0
123401085 115 4.0




 98%|██████████████████████████████████████████████████████████████████████████▍ | 18393/18770 [11:46<00:08, 43.87it/s]

 98%|██████████████████████████████████████████████████████████████████████████▍ | 18398/18770 [11:46<00:08, 44.17it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18403/18770 [11:46<00:08, 41.66it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18408/18770 [11:46<00:08, 40.91it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18413/18770 [11:46<00:08, 41.62it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18418/18770 [11:46<00:08, 41.86it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18423/18770 [11:46<00:08, 41.96it/s]

 98%|██████████████████████████████████████████████████████████████████████████▌ | 18428/18770 [11:46<00:08, 42.15it/s]

 98%|█████████████████████████

155016230 117 3.0
155016230 117 3.0




 99%|███████████████████████████████████████████████████████████████████████████ | 18553/18770 [11:49<00:05, 42.93it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18558/18770 [11:49<00:04, 43.45it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18563/18770 [11:50<00:04, 44.17it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18568/18770 [11:50<00:04, 44.28it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18573/18770 [11:50<00:04, 44.11it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18578/18770 [11:50<00:04, 43.94it/s]

 99%|███████████████████████████████████████████████████████████████████████████▏| 18583/18770 [11:50<00:04, 44.23it/s]

 99%|███████████████████████████████████████████████████████████████████████████▎| 18588/18770 [11:50<00:04, 44.85it/s]

 99%|█████████████████████████

In [95]:
df_result = pd.DataFrame(deemed_records)
df_result.columns = ['shopid','userid']
df_result['shopid'] = df_result['shopid'].astype(str)
df_result['userid'] = df_result['userid'].astype(str)
df_result.drop_duplicates(inplace=True)
df_result = df_result.groupby('shopid').agg({'userid': '&'.join}).reset_index()
df_result


,shopid,userid
0,100446829,2434757
1,10199219,8405753
2,10206302,95058664
3,103715156,214226569
4,10402,77819
...,...,...
252,97167904,214778616
253,98481320,124597967
254,98793086,142710562
255,99067259,108402614


In [96]:
test = df_result.copy()
df['shopid'] = df['shopid'].astype(str)
for s in tqdm(df.shopid.unique()):
    if s not in df_result.shopid.unique():
        test = test.append(pd.DataFrame([[s,0]],columns=['shopid','userid']), ignore_index=True)
        
test



  0%|                                                                                        | 0/18770 [00:00<?, ?it/s]

  0%|▎                                                                             | 85/18770 [00:00<00:22, 847.74it/s]

  1%|▋                                                                            | 154/18770 [00:00<00:23, 792.60it/s]

  1%|▉                                                                            | 229/18770 [00:00<00:23, 778.62it/s]

  2%|█▎                                                                           | 310/18770 [00:00<00:23, 787.09it/s]

  2%|█▌                                                                           | 391/18770 [00:00<00:23, 792.09it/s]

  3%|█▉                                                                           | 472/18770 [00:00<00:23, 795.10it/s]

  3%|██▎                                                                          | 554/18770 [00:00<00:22, 799.41it/s]

  3%|██▌                      

 51%|██████████████████████████████████████▊                                     | 9571/18770 [00:13<00:14, 626.64it/s]

 51%|███████████████████████████████████████                                     | 9634/18770 [00:13<00:14, 620.52it/s]

 52%|███████████████████████████████████████▎                                    | 9698/18770 [00:13<00:14, 622.98it/s]

 52%|███████████████████████████████████████▌                                    | 9761/18770 [00:14<00:14, 621.80it/s]

 52%|███████████████████████████████████████▊                                    | 9824/18770 [00:14<00:14, 621.89it/s]

 53%|████████████████████████████████████████                                    | 9888/18770 [00:14<00:14, 624.75it/s]

 53%|████████████████████████████████████████▎                                   | 9951/18770 [00:14<00:14, 618.36it/s]

 53%|████████████████████████████████████████                                   | 10015/18770 [00:14<00:14, 622.32it/s]

 54%|███████████████████████████

 94%|██████████████████████████████████████████████████████████████████████▏    | 17550/18770 [00:27<00:02, 538.17it/s]

 94%|██████████████████████████████████████████████████████████████████████▎    | 17604/18770 [00:27<00:02, 534.89it/s]

 94%|██████████████████████████████████████████████████████████████████████▌    | 17658/18770 [00:27<00:02, 528.88it/s]

 94%|██████████████████████████████████████████████████████████████████████▊    | 17713/18770 [00:27<00:01, 533.82it/s]

 95%|██████████████████████████████████████████████████████████████████████▉    | 17767/18770 [00:27<00:01, 531.93it/s]

 95%|███████████████████████████████████████████████████████████████████████▏   | 17823/18770 [00:27<00:01, 537.30it/s]

 95%|███████████████████████████████████████████████████████████████████████▍   | 17878/18770 [00:28<00:01, 540.62it/s]

 96%|███████████████████████████████████████████████████████████████████████▋   | 17933/18770 [00:28<00:01, 542.95it/s]

 96%|███████████████████████████

,shopid,userid
0,100446829,2434757
1,10199219,8405753
2,10206302,95058664
3,103715156,214226569
4,10402,77819
...,...,...
18765,212058195,0
18766,202216222,0
18767,208691471,0
18768,211987649,0


In [52]:
df_result.to_csv('partial_result.csv')

In [98]:
test.to_csv('200613_3rd_submission.csv', index=0)

In [97]:
test['userid'] = test['userid'].astype(str)
test[test['userid'].str.contains('&')]

,shopid,userid
60,136564914,191211430&178491887
120,175531295,187697407&215009429
124,181009364,214208720&101832161
149,201503467,130587573&92521144
190,51134277,29857724&212200633
191,51487211,5307816&214808165
193,54257623,1974334&107414154
243,8996761,162508227&13135622&215382704
